In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [2]:
weather = pd.read_csv("weather3_180703.csv")
weather['date'] = pd.to_datetime(weather["date"])
station = weather[weather['station_nbr'] == 18]

In [3]:
station['date'].count()

1035

In [4]:
# ls == 모든 값이 0인 것

ls = []
ls1 = []
for i in station.columns:
    if station[i].count() == 0:
        ls.append(i)
    if station[i].isna().sum() != 0:
        ls1.append(i)
ls, ls1

([], ['depart', 'sunrise', 'sunset', 'daytime'])

In [5]:
for i in station.columns:
    count_null = station[i].isna().sum()
    print(i, ":", count_null, "(",round((count_null / len(station) * 100),2),"%",")")

station_nbr : 0 ( 0.0 % )
date : 0 ( 0.0 % )
tmax : 0 ( 0.0 % )
tmin : 0 ( 0.0 % )
tavg : 0 ( 0.0 % )
depart : 31 ( 3.0 % )
dewpoint : 0 ( 0.0 % )
wetbulb : 0 ( 0.0 % )
heat : 0 ( 0.0 % )
cool : 0 ( 0.0 % )
sunrise : 31 ( 3.0 % )
sunset : 31 ( 3.0 % )
codesum : 0 ( 0.0 % )
snowfall : 0 ( 0.0 % )
preciptotal : 0 ( 0.0 % )
stnpressure : 0 ( 0.0 % )
sealevel : 0 ( 0.0 % )
resultspeed : 0 ( 0.0 % )
resultdir : 0 ( 0.0 % )
avgspeed : 0 ( 0.0 % )
year : 0 ( 0.0 % )
month : 0 ( 0.0 % )
day : 0 ( 0.0 % )
relative_humility : 0 ( 0.0 % )
windchill : 0 ( 0.0 % )
daytime : 31 ( 3.0 % )
week7 : 0 ( 0.0 % )
weekend : 0 ( 0.0 % )
code_change : 0 ( 0.0 % )
rainY : 0 ( 0.0 % )
otherY : 0 ( 0.0 % )
nothing : 0 ( 0.0 % )


In [6]:
train = pd.read_csv("train.csv")
train.date = pd.to_datetime(train.date)
train.tail()

,date,store_nbr,item_nbr,units
4617595,2014-10-31,45,107,0
4617596,2014-10-31,45,108,0
4617597,2014-10-31,45,109,0
4617598,2014-10-31,45,110,0
4617599,2014-10-31,45,111,0


In [7]:
key = pd.read_csv("key.csv")
station = station.merge(key)

In [8]:
station = station.merge(train)
station.tail()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr,units
106999,18,2014-10-31,67.0,49.0,58.0,-4.0,40.0,50.0,7.0,0.0,...,10.9,4,0,0,0,0,1,36,107,0
107000,18,2014-10-31,67.0,49.0,58.0,-4.0,40.0,50.0,7.0,0.0,...,10.9,4,0,0,0,0,1,36,108,0
107001,18,2014-10-31,67.0,49.0,58.0,-4.0,40.0,50.0,7.0,0.0,...,10.9,4,0,0,0,0,1,36,109,0
107002,18,2014-10-31,67.0,49.0,58.0,-4.0,40.0,50.0,7.0,0.0,...,10.9,4,0,0,0,0,1,36,110,0
107003,18,2014-10-31,67.0,49.0,58.0,-4.0,40.0,50.0,7.0,0.0,...,10.9,4,0,0,0,0,1,36,111,0


In [9]:
station['log1p_units'] = np.log1p(station.units)

In [10]:
target1 = station['units']
target2 = station['log1p_units']

station.drop(columns=['units','log1p_units'],inplace=True)
station.tail()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,windchill,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr
106999,18,2014-10-31,67.0,49.0,58.0,-4.0,40.0,50.0,7.0,0.0,...,55.662463,10.9,4,0,0,0,0,1,36,107
107000,18,2014-10-31,67.0,49.0,58.0,-4.0,40.0,50.0,7.0,0.0,...,55.662463,10.9,4,0,0,0,0,1,36,108
107001,18,2014-10-31,67.0,49.0,58.0,-4.0,40.0,50.0,7.0,0.0,...,55.662463,10.9,4,0,0,0,0,1,36,109
107002,18,2014-10-31,67.0,49.0,58.0,-4.0,40.0,50.0,7.0,0.0,...,55.662463,10.9,4,0,0,0,0,1,36,110
107003,18,2014-10-31,67.0,49.0,58.0,-4.0,40.0,50.0,7.0,0.0,...,55.662463,10.9,4,0,0,0,0,1,36,111


In [11]:
df1 = pd.concat([station,target1], axis=1)
df2 = pd.concat([station,target2], axis=1)

In [12]:
df2.to_csv('station18.csv', sep=',', index=False)

In [13]:
if df2.columns.tolist() and ls:
    print('공통 변수 있음')
    print(df2.columns.tolist() and ls)
else:
    print('공통 변수 없음')

공통 변수 없음


In [14]:
df2.columns

Index(['station_nbr', 'date', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint',
       'wetbulb', 'heat', 'cool', 'sunrise', 'sunset', 'codesum', 'snowfall',
       'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
       'avgspeed', 'year', 'month', 'day', 'relative_humility', 'windchill',
       'daytime', 'week7', 'weekend', 'code_change', 'rainY', 'otherY',
       'nothing', 'store_nbr', 'item_nbr', 'log1p_units'],
      dtype='object')

In [15]:
ls, ls1

([], ['depart', 'sunrise', 'sunset', 'daytime'])

### 1. 변수변환: df2 (log1p_units)

In [17]:
all_var = 'log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(depart) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                            + scale(sunrise) + scale(sunset) + scale(snowfall) + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + C(resultdir) + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + C(daytime) + 0'

In [18]:
model2 = sm.OLS.from_formula('log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                           + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + C(resultdir) + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2)
result2 = model2.fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.960
Model:                            OLS   Adj. R-squared:                  0.960
Method:                 Least Squares   F-statistic:                 1.484e+04
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:47:17   Log-Likelihood:                 90292.
No. Observations:              107004   AIC:                        -1.802e+05
Df Residuals:                  106830   BIC:                        -1.786e+05
Df Model:                         173                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(resultdir)[1.0]       

### 2. 변수변환 : df2 (log1p_units) + 아웃라이어 제거

In [19]:
# 아웃라이어 제거
# Cook's distance > 2 인 값 제거
influence = result2.get_influence()
cooks_d2, pvals = influence.cooks_distance
fox_cr = 4 / (len(df2) - 2)
idx_outlier = np.where(cooks_d2 > fox_cr)[0]
len(idx_outlier)

2893

In [20]:
idx = list(set(range(len(df2))).difference(idx_outlier))
df2_1 = df2.iloc[idx, :].reset_index(drop=True)
df2_1

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr,log1p_units
0,18,2012-01-01,59.0,40.0,49.5,4.0,28.0,40.0,15.0,0.0,...,10.033333,6,1,0,0,0,1,36,2,0.0
1,18,2012-01-01,59.0,40.0,49.5,4.0,28.0,40.0,15.0,0.0,...,10.033333,6,1,0,0,0,1,36,3,0.0
2,18,2012-01-01,59.0,40.0,49.5,4.0,28.0,40.0,15.0,0.0,...,10.033333,6,1,0,0,0,1,36,4,0.0
3,18,2012-01-01,59.0,40.0,49.5,4.0,28.0,40.0,15.0,0.0,...,10.033333,6,1,0,0,0,1,36,5,0.0
4,18,2012-01-01,59.0,40.0,49.5,4.0,28.0,40.0,15.0,0.0,...,10.033333,6,1,0,0,0,1,36,6,0.0
5,18,2012-01-01,59.0,40.0,49.5,4.0,28.0,40.0,15.0,0.0,...,10.033333,6,1,0,0,0,1,36,7,0.0
6,18,2012-01-01,59.0,40.0,49.5,4.0,28.0,40.0,15.0,0.0,...,10.033333,6,1,0,0,0,1,36,8,0.0
7,18,2012-01-01,59.0,40.0,49.5,4.0,28.0,40.0,15.0,0.0,...,10.033333,6,1,0,0,0,1,36,10,0.0
8,18,2012-01-01,59.0,40.0,49.5,4.0,28.0,40.0,15.0,0.0,...,10.033333,6,1,0,0,0,1,36,11,0.0
9,18,2012-01-01,59.0,40.0,49.5,4.0,28.0,40.0,15.0,0.0,...,10.033333,6,1,0,0,0,1,36,12,0.0


In [21]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                           + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + C(resultdir) + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 5.660e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:48:06   Log-Likelihood:             3.3254e+05
No. Observations:              104111   AIC:                        -6.647e+05
Df Residuals:                  103938   BIC:                        -6.631e+05
Df Model:                         172                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(resultdir)[1.0]       

In [22]:
ls1

['depart', 'sunrise', 'sunset', 'daytime']

### 3. VIF 확인

In [23]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# all_cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
#         'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
#         'windchill', 'depart', 'sunrise', 'sunset', 'snowfall' ]
cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
        'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
        'windchill', 'snowfall' ]

y = df2_1.loc[:,cols]
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(y.values, i) for i in range(y.shape[1])]
vif["features"] = y.columns
vif = vif.sort_values("VIF Factor", ascending=False).reset_index(drop=True)
vif

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:167: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,VIF Factor,features
0,inf,tmax
1,inf,tmin
2,inf,tavg
3,2.821202e+06,sealevel
4,2.799724e+06,stnpressure
5,1.137742e+04,windchill
6,4.387650e+03,wetbulb
7,3.613400e+03,dewpoint
8,8.377288e+02,relative_humility
9,1.390717e+02,cool


### 4. VIF 근거, 빼고 다시 Model 돌림.

In [24]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(heat)\
                            + scale(snowfall) + scale(preciptotal) + scale(resultspeed)  \
                            + scale(avgspeed) + C(year) + C(month) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 7.548e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:48:51   Log-Likelihood:             3.3253e+05
No. Observations:              104111   AIC:                        -6.648e+05
Df Residuals:                  103981   BIC:                        -6.635e+05
Df Model:                         129                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(year)[2012]          0.0002      0

### 5. VIF 재확인 

In [25]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# all_cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
#         'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
#         'windchill', 'depart', 'sunrise', 'sunset', 'snowfall' ]
cols = ['heat', 'snowfall', 'preciptotal', 'resultspeed', 'avgspeed']

y = df2_1.loc[:,cols]
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(y.values, i) for i in range(y.shape[1])]
vif["features"] = y.columns
vif = vif.sort_values("VIF Factor", ascending=False).reset_index(drop=True)
vif

,VIF Factor,features
0,22.347568,avgspeed
1,21.477764,resultspeed
2,1.280790,heat
3,1.096328,preciptotal
4,1.053606,snowfall


### 6. VIF 근거로 변수 제외하고  Model 돌림.

In [26]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(heat)\
                            + scale(snowfall) + scale(preciptotal) + C(year) + C(month) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 7.667e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:49:22   Log-Likelihood:             3.3252e+05
No. Observations:              104111   AIC:                        -6.648e+05
Df Residuals:                  103983   BIC:                        -6.636e+05
Df Model:                         127                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(year)[2012]          0.0003      0

### Anova 분석을 통해 p-value가 0.05보다 큰것은 제외

In [27]:
anova_result2_1 = sm.stats.anova_lm(result2_1).sort_values(by=['PR(>F)'], ascending = False)
anova_result2_1[anova_result2_1['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
C(weekend),1.0,0.005178,0.005178,52.523995,4.279582e-13
C(rainY),1.0,0.011315,0.011315,114.771180,9.121284e-27
C(year),3.0,71.555399,23.851800,241928.173784,0.000000e+00
C(month),11.0,0.744419,0.067674,686.420170,0.000000e+00
C(item_nbr),109.0,9596.341595,88.039831,892985.676038,0.000000e+00


### 최종 Anova 분석과 TIF 통과한 변수들을 바탕으로 최종 모델생성

In [28]:
# OLS - df2_1_1
model2_1 = sm.OLS.from_formula('log1p_units ~ C(rainY) + C(year) + C(month) + \
+ C(item_nbr) + C(weekend) + 0', data = df2_1)

result = model2_1.fit()
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 7.852e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:50:29   Log-Likelihood:             3.3252e+05
No. Observations:              104111   AIC:                        -6.648e+05
Df Residuals:                  103986   BIC:                        -6.636e+05
Df Model:                         124                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(rainY)[0]            0.0002      0

In [29]:
from patsy import dmatrix
# 독립변수와 종속변수로 나누기
df2_1_X = df2_1.drop(columns=['log1p_units'])
df2_1_target = df2_1['log1p_units']

In [32]:
formula = 'C(rainY) + C(year) + C(month) + \
+ C(item_nbr) + C(weekend) + 0'
dfX = dmatrix(formula, df2_1_X, return_type='dataframe')
dfy = pd.DataFrame(df2_1_target, columns=["log1p_units"])

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

model = LinearRegression()
cv = KFold(10)

scores = np.zeros(10)
for i, (train_index, test_index) in enumerate(cv.split(dfX)):
    X_train = dfX.values[train_index]
    y_train = dfy.values[train_index]
    X_test = dfX.values[test_index]
    y_test = dfy.values[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores[i] = r2_score(y_test, y_pred)

scores

array([0.99879276, 0.99875213, 0.99914632, 0.99913478, 0.99881765,
       0.99902847, 0.99906349, 0.99875351, 0.99863225, 0.99866042])